# Querying Coinbase Transaction with Libbitcoin

In [ ]:
// Compiler & linker information for c++ interpreter.
#pragma cling add_include_path("/usr/local/include","/usr/local/Cellar/zeromq/4.2.5/include")
#pragma cling add_library_path("/usr/local/lib","/usr/local/Cellar/zeromq/4.2.5/lib")
#pragma cling load("bitcoin","bitcoin-protocol","zmq","secp256k1","pthread","boost_chrono-mt","boost_date_time-mt","boost_filesystem","boost_iostreams-mt","boost_locale-mt","boost_log-mt","boost_program_options-mt","boost_regex-mt","boost_system","boost_thread-mt")

// Required header files.
#include <iostream>
#include <string>
#include <bitcoin/protocol.hpp> // Libbitcoin-Protocol

## Fetch transaction hashes & coinbase transaction data
* Server request for all block `tx_hashes` at `height`
* Server request for coinbase data with txid: `tx_hashes[0]`

In [21]:
{
    // Setup of zmq context & socket.
    bc::protocol::zmq::context my_context(true); //started
    bc::protocol::zmq::socket dealer_socket(
        my_context, bc::protocol::zmq::socket::role::dealer);
    bc::code ec;
    bc::config::endpoint public_endpoint("tcp://mainnet1.libbitcoin.net:9091");
    ec = dealer_socket.connect(public_endpoint);

    //--------------------------------------------------------------------------
    // 1) Request block_tx_hashes.
    //--------------------------------------------------------------------------

    bc::protocol::zmq::message block_tx_hashes_request;

    std::string block_tx_hashes_command =
        "blockchain.fetch_block_transaction_hashes";
    uint32_t message_id0(0);
    uint32_t height(422108);

    block_tx_hashes_request.enqueue(bc::to_chunk(block_tx_hashes_command));
    block_tx_hashes_request.enqueue(bc::to_chunk(bc::to_little_endian(message_id0)));
    block_tx_hashes_request.enqueue_little_endian(height);

    if ((ec = block_tx_hashes_request.send(dealer_socket)))
        {
        std::cout << ec.message() << std::endl;
        return 1;
        }

    std::vector<bc::hash_digest> tx_hashes;

    bc::protocol::zmq::poller my_poller;
    my_poller.add(dealer_socket); 
    auto dealer_socket_id = dealer_socket.id();
    auto socket_ids = my_poller.wait(2000);

    if (socket_ids.contains(dealer_socket_id))
    {
        bc::protocol::zmq::message server_response;
        uint32_t my_message_id;
        bc::data_chunk reply_payload;

        server_response.receive(dealer_socket);
        
        std::string my_message_block_tx_hashes_command =
            server_response.dequeue_text();
        server_response.dequeue(my_message_id);
        server_response.dequeue(reply_payload);

        bc::data_source reply_byte_stream(reply_payload);
        bc::istream_reader reply_byte_stream_reader(reply_byte_stream);

        if ((ec = reply_byte_stream_reader.read_error_code())) {
            std::cout << ec.message() << std::endl;
            return 1;
        }

        while (!reply_byte_stream_reader.is_exhausted())
        {
            tx_hashes.push_back(reply_byte_stream_reader.read_hash());
        }
    }

    else
    {
      std::cout << "Request timed out..." << std::endl;
    }
    
    //--------------------------------------------------------------------------
    // 2) Request coinbase data with txid tx_hashes[0].
    //--------------------------------------------------------------------------

    bc::protocol::zmq::message tx_request;
    std::string tx_command = "blockchain.fetch_transaction2";
    uint32_t message_id1(1);

    tx_request.enqueue(bc::to_chunk(tx_command));
    tx_request.enqueue(bc::to_chunk(bc::to_little_endian(message_id1)));
    tx_request.enqueue(bc::to_chunk(tx_hashes[0]));

    if ((ec = tx_request.send(dealer_socket)))
    {
        std::cout << ec.message() << std::endl;
        return 1;
    }

    // Poll our socket.
    socket_ids = my_poller.wait(5000);
    if (socket_ids.contains(dealer_socket_id))
    {
        bc::protocol::zmq::message server_response;
        uint32_t my_message_id;
        bc::data_chunk reply_payload;

        server_response.receive(dealer_socket);

        std::string my_message_block_tx_hashes_command =
            server_response.dequeue_text();
        server_response.dequeue(my_message_id);
        server_response.dequeue(reply_payload);

        bc::data_source reply_byte_stream(reply_payload);
        bc::istream_reader reply_byte_stream_reader(reply_byte_stream);

        if ((ec = reply_byte_stream_reader.read_error_code())) {
            std::cout << ec.message() << std::endl;
            return 1;
        }

        bc::chain::transaction coinbase;
        // wire=true, witness=true
        coinbase.from_data(reply_byte_stream_reader, true, true);
        
        auto coinbase_txid = coinbase.hash();
        std::reverse(std::begin(coinbase_txid), std::end(coinbase_txid));
        
        std::cout << "Coinbase TXID: "
                  << bc::encode_base16(bc::to_chunk(coinbase_txid)) 
                  << "\n" << std::endl;
        
        std::cout <<"Raw coinbase transaction: "
                  << bc::encode_base16(coinbase.to_data(true, true)) 
                  << "\n" << std::endl;

        // Check fees and subsidy computation.
        const auto subsidy = bc::chain::block::subsidy(height, true); // independent of coinbase.
        const auto fees = coinbase.total_output_value() - subsidy;

        std::cout << "Block subsidy: "
                  << bc::encode_base10(subsidy) << "\n" << std::endl;
        std::cout << "Block fees: "
                  << bc::encode_base10(fees) << "\n" << std::endl;
    }

    else
    {
        std::cout << "Request timed out..." << std::endl;
    }
  
} // End of jupyter cell scope.

Coinbase TXID: bf846c7694995e805b4e917c6f4c4fb8ef06bc588e2ae17b2f1d57b98b43dd18

Raw coinbase transaction: 01000000010000000000000000000000000000000000000000000000000000000000000000ffffffff6403dc700637e4b883e5bda9e7a59ee4bb99e9b1bc12f4d4be6b4fec3398bab6fd2e81e7705d21d2f95f196911c8f3c6184b34adc902000000f09f909f0e4d696e656420627920636363363400000000000000000000000000000000000000000000000000cbc4010001ff11824b000000001976a914c825a1ecf2a6830c4401620c3a16f1995057c2ab88ac2c4aa338

Block subsidy: 1250000000

Block fees: 16815487



### Coinbase transaction lookup with BX.

In [ ]:
!bx fetch-tx bf846c7694995e805b4e917c6f4c4fb8ef06bc588e2ae17b2f1d57b98b43dd18


In [ ]:
!sfk dec 0670dc